In [1]:
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# Create a console handler and set its level
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Create a formatter and set it on the console handler
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)

# Add the console handler to the logger
logger.addHandler(console_handler)

def main():
    logger.info("Cloud function starting.")
    # 1. Scrape coingecko gainers.
    import scraping.scraper
    scraping.scraper.main()

    # 2. Get contract address for gainers that have them, with coingecko.
    import scraping.contract_addresses
    scraping.contract_addresses.main()

    # 3. Run SQL query on Google Big Query to find wallets that have receieved these tokens within the last 7 days.
    
main()

2023-03-31 16:29:27,194 - INFO - Cloud function starting.
2023-03-31 16:29:28 [scrapy.utils.log] INFO: Scrapy 2.8.0 started (bot: scrapybot)
2023-03-31 16:29:28 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.13, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.11.1 (v3.11.1:a7a450f84a, Dec  6 2022, 15:24:06) [Clang 13.0.0 (clang-1300.0.29.30)], pyOpenSSL 23.1.1 (OpenSSL 3.1.0 14 Mar 2023), cryptography 40.0.1, Platform macOS-13.2.1-arm64-arm-64bit
2023-03-31 16:29:28 [scrapy.crawler] INFO: Overridden settings:
{}
2023-03-31 16:29:28 [py.warnings] WARNING: /Users/dbless/Documents/Github/emperor-traders/.venv/lib/python3.11/site-packages/scrapy/utils/request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for

In [32]:
import pandas as pd

with open('gainers.csv') as f:
    df = pd.read_csv(f)

token_contracts = [f"'{contract}'" for contract in df['contract_address']]
token_contracts_string = f"({','.join(token_contracts)})"

In [33]:
token_contracts_string

"('0x095797fd4297fb79883cc912a5ba6313b15c445d','0xe41d2489571d322189246dafa5ebde1f4699f498','0x55c08ca52497e2f1534b59e2917bf524d4765257','0xdb85f6685950e285b1e611037bebe5b34e2b7d78','0x6bea7cfef803d1e3d5f7c0103f7ded065644e197','0xec213f83defb583af3a000b1c0ada660b1902a0f','0x51cb253744189f11241becb29bedd3f1b5384fdb','0x6368e1e18c4c419ddfc608a0bed1ccb87b9250fc','0xa670d7237398238de01267472c6f13e5b8010fd1','0xafcdd4f666c84fed1d8bd825aa762e3714f652c9','0x3a856d4effa670c54585a5d523e96513e148e95d','0x5283d291dbcf85356a21ba090e6db59121208b44','0x8e6cd950ad6ba651f6dd608dc70e5886b1aa6b24','0xf4d2888d29d722226fafa5d9b24f9164c092421e','0xae12c5930881c53715b369cec7606b70d8eb229f','0x1e4ede388cbc9f4b5c79681b7f94d36a11abebc9','0x090185f2135308bad17527004364ebcc2d37e5f6','0x8ce9137d39326ad0cd6491fb5cc0cba0e089b6a9','0x3affcca64c2a6f4e3b6bd9c64cd2c969efd1ecbe','0x701c244b988a513c945973defa05de933b23fe1d')"

In [13]:
import os

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv() 
KEY_PATH = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
PROJECT_ID = os.getenv("PROJECT_ID")

from google.cloud import bigquery
from google.oauth2 import service_account


# Authenticate to BigQuery
credentials = service_account.Credentials.from_service_account_file(
    KEY_PATH, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

dry_run_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
wet_run_config = bigquery.QueryJobConfig(use_query_cache=False)

# Query the last 7 days of token transfers
query = f"""
SELECT DISTCINCT tt.from_address
FROM `bigquery-public-data.crypto_ethereum.token_transfers` tt
WHERE tt.block_timestamp > TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY) 
AND tt.block_timestamp < TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 DAY) 
AND tt.token_address IN {token_contracts_string}
LIMIT 10 
"""

logger.info("Running SQL query on Google Big Cloud.")
dry_query = client.query(query, job_config=dry_run_config)

# Check how much data will be processed
mb_processed = dry_query.total_bytes_processed / (1024 * 1024)
# Throw error if over 1024 MB
if mb_processed > 1024:
    logger.error(f"Query will process {mb_processed:.2f} MB.")
    raise Exception(f"Query will process {mb_processed:.2f} MB.")

logger.info(f"Query will process {mb_processed:.0f} MB.")


wet_query = client.query(query, job_config=wet_run_config)
df = wet_query.to_dataframe()

2023-03-31 16:32:07,574 - INFO - Running SQL query on Google Big Cloud.
2023-03-31 16:32:10,403 - INFO - Query will process 332 MB.


In [14]:
df

,token_address,from_address,to_address,value,transaction_hash,log_index,block_timestamp,block_number,block_hash
0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x1111111254eeb25477b68fb85ed929f73a960582,0xc37ecc839ba67a46847dc367252b057f8feae680,495625000000000000,0x4708c3ac79b2e728677fffa664a098fcfb5c3119005d...,61,2023-03-31 22:32:23+00:00,16950171,0x46e91b2e94c8034db439f43cf2b6e6553b22735b1b46...
1,0x429881672b9ae42b8eba0e26cd9c73711b891ca5,0xda481b277dce305b97f4091bd66595d57cf31634,0xa01f81a76e11a00f130c7eca21cfd1c4edc5d84a,111112447622811259261,0xdb7961fcde2f6b92c5a8b96f49f5f85f29eb7600fd0d...,170,2023-03-31 22:32:23+00:00,16950171,0x46e91b2e94c8034db439f43cf2b6e6553b22735b1b46...
2,0x2bd938cf96430b7b0879f76b010b589aeec2127c,0x0000000000000000000000000000000000000000,0xad4db3604a2d5867a5c9b9a62908e94ad5d16871,107761,0xe7953187b5a99251c69a7aafa07aebd5b18cbc54209b...,229,2023-03-31 22:32:23+00:00,16950171,0x46e91b2e94c8034db439f43cf2b6e6553b22735b1b46...
3,0xdac17f958d2ee523a2206206994597c13d831ec7,0x68a1e07ac7850c3f10c00bf443a1bad6835b4b47,0xcf03463419f34eb34d9c74b51adc7b95060deaef,61373509,0xb3d62e910b32dff4df53772a298845e7506d268599ac...,134,2023-03-31 22:32:23+00:00,16950171,0x46e91b2e94c8034db439f43cf2b6e6553b22735b1b46...
4,0x9665b00ce87cc5a51869095c38b2c77313a18a3b,0x4aa087a88dc3fa72b17edb1f9b284f21e369afad,0x07c42831b30c68824f261c61074ac13f8aa885d3,12,0x64063c839f2043731247990d07b2989eee4462206e36...,104,2023-03-31 22:32:23+00:00,16950171,0x46e91b2e94c8034db439f43cf2b6e6553b22735b1b46...
5,0xe22910d04bdc9d55570f3bc52e786b49dddf37b4,0x74de5d4fcbf63e00296fd95d33236b9794016631,0x72977908f6101e84c813634ff6d796e488570896,687649363975849972452,0x4708c3ac79b2e728677fffa664a098fcfb5c3119005d...,66,2023-03-31 22:32:23+00:00,16950171,0x46e91b2e94c8034db439f43cf2b6e6553b22735b1b46...
6,0x22c83c0e004c70b22a41e2655574764e1871e9af,0x68ca247510221ce94c0096229da8031e7b2b7ee1,0x6cdbef8caaee41b69386ae78d9f8e729e652017b,141,0x2a9a1d7e24c0f3506f3acf8e4379c40052773a6b432d...,184,2023-03-31 22:32:23+00:00,16950171,0x46e91b2e94c8034db439f43cf2b6e6553b22735b1b46...
7,0x2bd938cf96430b7b0879f76b010b589aeec2127c,0x0000000000000000000000000000000000000000,0xad4db3604a2d5867a5c9b9a62908e94ad5d16871,107759,0xe7953187b5a99251c69a7aafa07aebd5b18cbc54209b...,227,2023-03-31 22:32:23+00:00,16950171,0x46e91b2e94c8034db439f43cf2b6e6553b22735b1b46...
8,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xc37ecc839ba67a46847dc367252b057f8feae680,0x6b75d8af000000e20b7a7ddf000ba900b4009a80,301119584011478656,0xe10e1eb3f4322f4bf1485de1a066c943cebb0fc0537e...,69,2023-03-31 22:32:23+00:00,16950171,0x46e91b2e94c8034db439f43cf2b6e6553b22735b1b46...
9,0x565a21cd4471287e25f2b85d14d569db03b63a8f,0xc8942183b626408198be7dec512a1e68deb6319d,0x05a8aa0ed1e1bc598c23b415f67cd774b530546c,887,0x7fb0772148244db838956e5b14e84bde5a30303fbe82...,258,2023-03-31 22:32:23+00:00,16950171,0x46e91b2e94c8034db439f43cf2b6e6553b22735b1b46...


1048576

In [4]:
wet_query.to_dataframe()

,name,total_people
0,James,272793
1,John,235139
2,Michael,225320
3,Robert,220399
4,David,219028
5,Mary,209893
6,William,173092
7,Jose,157362
8,Christopher,144196
9,Maria,131056
